Note that it might make more sense to just request cov(x,y), std(x), std(y) and compute correlation on requester's end instead of requesting the whole HUGE circuit of correlation. 

In [1]:
import ezkl
import torch
from torch import nn
import json
import os
import time
import scipy
import numpy as np
import matplotlib.pyplot as plt
import statistics
import math

In [2]:
from zkstats.core import create_dummy, verifier_define_calculation, prover_gen_settings, setup, prover_gen_proof, verifier_verify, get_data_commitment_maps

In [3]:
# init path
os.makedirs(os.path.dirname('shared/'), exist_ok=True)
os.makedirs(os.path.dirname('prover/'), exist_ok=True)
verifier_model_path = os.path.join('shared/verifier.onnx')
prover_model_path = os.path.join('prover/prover.onnx')
verifier_compiled_model_path = os.path.join('shared/verifier.compiled')
prover_compiled_model_path = os.path.join('prover/prover.compiled')
pk_path = os.path.join('shared/test.pk')
vk_path = os.path.join('shared/test.vk')
proof_path = os.path.join('shared/test.pf')
settings_path = os.path.join('shared/settings.json')
srs_path = os.path.join('shared/kzg.srs')
witness_path = os.path.join('prover/witness.json')
# this is private to prover since it contains actual data
sel_data_path = os.path.join('prover/sel_data.json')
# this is just dummy random value
sel_dummy_data_path = os.path.join('shared/sel_dummy_data.json')

=======================  ZK-STATS FLOW =======================

This example is not necessary. In fact, a person can just request cov(x,y), std(x), and std(y). Then just compute correlation on his own as well, but here we show that the code is composable enough to do all at once. 

In [4]:
data_path = os.path.join('data.json')
dummy_data_path = os.path.join('shared/dummy_data.json')

create_dummy(data_path, dummy_data_path)

In [5]:
scales = [5]
selected_columns = ['x', 'y']
commitment_maps = get_data_commitment_maps(data_path, scales)

In [6]:
# Verifier/ data consumer side: send desired calculation
from zkstats.computation import computation_to_model, State


def computation(s: State, data: list[torch.Tensor]) -> torch.Tensor:
    x, y = data[0], data[1]
    filter =(y<60)
    filtered_x = s.where(filter, x)
    filtered_y = s.where(filter, y)
    return s.correlation(filtered_x, filtered_y)

error = 0.01
_, verifier_model = computation_to_model(computation, error)

verifier_define_calculation(dummy_data_path, selected_columns, sel_dummy_data_path, verifier_model, verifier_model_path)

/Users/jernkun/Desktop/zk-stats-lib/zkstats/computation.py:172: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  is_precise_aggregated = torch.tensor(1.0)
/Users/jernkun/Library/Caches/pypoetry/virtualenvs/zkstats-OJpceffF-py3.11/lib/python3.11/site-packages/torch/onnx/symbolic_opset9.py:2174: FutureWarning: 'torch.onnx.symbolic_opset9._cast_Bool' is deprecated in version 2.0 and will be removed in the future. Please Avoid using this function and create a Cast node instead.
  return fn(g, to_cast_func(g, input, False), to_cast_func(g, other, False))


In [7]:
# Prover/ data owner side
_, prover_model = computation_to_model(computation, error)

prover_gen_settings(data_path, selected_columns, sel_data_path, prover_model,prover_model_path, scales, "resources", settings_path)



 <------------- Numerical Fidelity Report (input_scale: 5, param_scale: 5, scale_input_multiplier: 10) ------------->

+---------------+---------------+-----------+---------------+----------------+------------------+---------------+---------------+--------------------+--------------------+------------------------+
| mean_error    | median_error  | max_error | min_error     | mean_abs_error | median_abs_error | max_abs_error | min_abs_error | mean_squared_error | mean_percent_error | mean_abs_percent_error |
+---------------+---------------+-----------+---------------+----------------+------------------+---------------+---------------+--------------------+--------------------+------------------------+
| -0.0014694929 | -0.0029389858 | 0         | -0.0029389858 | 0.0014694929   | 0.0029389858     | 0.0029389858  | 0             | 0.000004318819     | -0.0027814922      | 0.0027814922           |
+---------------+---------------+-----------+---------------+----------------+-------------

==== Generate & Calibrate Setting ====
scale:  [5]
setting:  {"run_args":{"tolerance":{"val":0.0,"scale":1.0},"input_scale":5,"param_scale":5,"scale_rebase_multiplier":10,"lookup_range":[-2292,3712],"logrows":13,"num_inner_cols":2,"variables":[["batch_size",1]],"input_visibility":{"Hashed":{"hash_is_public":true,"outlets":[]}},"output_visibility":"Public","param_visibility":"Private","div_rebasing":false,"rebase_frac_zero_constants":false,"check_mode":"UNSAFE"},"num_rows":7872,"total_assignments":1517,"total_const_size":308,"model_instance_shapes":[[1],[1]],"model_output_scales":[0,5],"model_input_scales":[5,5],"module_sizes":{"kzg":[],"poseidon":[7872,[2]]},"required_lookups":[{"GreaterThan":{"a":0.0}}],"required_range_checks":[],"check_mode":"UNSAFE","version":"9.1.0","num_blinding_factors":null,"timestamp":1709724737371}


In [8]:
# Here verifier & prover can concurrently call setup since all params are public to get pk.
# Here write as verifier function to emphasize that verifier must calculate its own vk to be sure
setup(verifier_model_path, verifier_compiled_model_path, settings_path,vk_path, pk_path )

print("=======================================")
# Prover generates proof
prover_gen_proof(prover_model_path, sel_data_path, witness_path, prover_compiled_model_path, settings_path, proof_path, pk_path)

==== setting up ezkl ====
Time setup: 0.8491756916046143 seconds
==== Generating Witness ====
witness boolean:  1.0
witness result 1 : 0.53125
==== Generating Proof ====
proof:  {'instances': [['7469705c7b45875730aac31b009e8f9e9f9e57031bba148110877f78e4518c22', 'c1f946e3fd2b2229bfcc2dc9ff7f4dfc1fddad53635becb58d130bcae5504712', '0100000000000000000000000000000000000000000000000000000000000000', '1100000000000000000000000000000000000000000000000000000000000000']], 'proof': '0x03aaee71028b0e3f9e18e95faf91a25a27c23169df088238d03951419aa5fad90268273f3b2027430b5d5f1ff30b28e6d1d858c9e44efece6aa8bf25a0d5ee991a3c55a457cc167a914f4c83a33a65998525fa1e8c7a2b4e93ccd22c1a6a210904a0d1cce495ec2cb43ee49e063c1f6a220a919673fc0b7cc8b0f9e2f3af29af2023427489ce5479be74a14df012805bc9a1c2e1a4c541039c05c013f864d2471cc5401462d65b230efc7c63a32c9544b4fe8564a6e3c2e479c938a10fd1f13d025e28b676518ce24f0b8018f4c7c72a798ad9944911bf0b862f84147a88285d2f0776efbbeaef6134f99fe04ce6e5080df3daaec2c596dc471a365778a9799d1242f8bd

In [9]:
# Verifier verifies
res = verifier_verify(proof_path, settings_path, vk_path, selected_columns, commitment_maps)
print("Verifier gets result:", res)

Verifier gets result: [0.53125]
